In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")

print("Path to dataset files:", path)

d:\Ao\Code\AI\deep-learning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 331M/331M [11:33<00:00, 501kB/s] 

Extracting files...


Path to dataset files: C:\Users\Mr Sagar Kumar\.cache\kagglehub\datasets\jangedoo\utkface-new\versions\1


In [4]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator    

In [30]:
folder_path = r"C:\Users\Mr Sagar Kumar\.cache\kagglehub\datasets\jangedoo\utkface-new\versions\1\UTKFace"

In [102]:
# Reading images and creating a df
age = []
gender = []
img_path = []
for file in os.listdir(folder_path):
    ls = file.split("_")
    age.append(ls[0])
    gender.append(ls[1])
    img_path.append(file)


In [103]:
df = pd.DataFrame({"age": age, "gender":gender, "img":img_path})
df
# Image on index 0: 100_0_0_20170112213500903.jpg.chip.jpg; In the image path 100 is age and the 0 after 100 is the gender

,age,gender,img
0,100,0,100_0_0_20170112213500903.jpg.chip.jpg
1,100,0,100_0_0_20170112215240346.jpg.chip.jpg
2,100,1,100_1_0_20170110183726390.jpg.chip.jpg
3,100,1,100_1_0_20170112213001988.jpg.chip.jpg
4,100,1,100_1_0_20170112213303693.jpg.chip.jpg
...,...,...,...
23703,9,1,9_1_3_20161220222856346.jpg.chip.jpg
23704,9,1,9_1_3_20170104222949455.jpg.chip.jpg
23705,9,1,9_1_4_20170103200637399.jpg.chip.jpg
23706,9,1,9_1_4_20170103200814791.jpg.chip.jpg


In [104]:
df['age'] = df['age'].astype(np.float32)
df['gender'] = df['gender'].astype(np.float32)


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23708 entries, 0 to 23707
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     23708 non-null  float32
 1   gender  23708 non-null  float32
 2   img     23708 non-null  object 
dtypes: float32(2), object(1)
memory usage: 370.6+ KB


In [106]:
# train_df = df.sample(frac=1,random_state=0).iloc[:20000]
# test_df = df.sample(frac=1,random_state=0).iloc[20000:]

# Then train/test split
train_df = df.sample(frac=1, random_state=0).iloc[:20000]
test_df = df.sample(frac=1, random_state=0).iloc[20000:]

In [107]:
train_df.shape

(20000, 3)

In [108]:
test_df.shape

(3708, 3)

## Data Augmentation

In [124]:
# 2. Data Generators with Proper Output Signature
def create_generator(dataframe, directory, batch_size=32, is_train=True):
    if is_train:
        datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True
        )
    else:
        datagen = ImageDataGenerator(rescale=1./255)
    
    generator = datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='img',
        y_col=['age', 'gender'],
        target_size=(200, 200),
        batch_size=batch_size,
        class_mode='multi_output'
    )
    
    # Convert to tf.data.Dataset with proper output signature
    output_signature = (
        tf.TensorSpec(shape=(None, 200, 200, 3)),  # Images
        (
            tf.TensorSpec(shape=(None, 1)),  # Age
            tf.TensorSpec(shape=(None, 1))   # Gender
        )
    )
    
    dataset = tf.data.Dataset.from_generator(
        lambda: generator,
        output_signature=output_signature
    )
    
    return dataset

train_dataset = create_generator(train_df, folder_path, is_train=True)
test_dataset = create_generator(test_df, folder_path, is_train=False)

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [125]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [126]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [121]:
# 3. Model Architecture
base_model = ResNet50(include_top=False, input_shape=(200, 200, 3))
base_model.trainable = False

inputs = Input(shape=(200, 200, 3))
x = base_model(inputs, training=False)
x = Flatten()(x)

# Age branch
age_branch = Dense(512, activation='relu')(x)
age_branch = Dense(512, activation='relu')(age_branch)
age_output = Dense(1, activation='linear', name='age')(age_branch)

# Gender branch
gender_branch = Dense(512, activation='relu')(x)
gender_branch = Dense(512, activation='relu')(gender_branch)
gender_output = Dense(1, activation='sigmoid', name='gender')(gender_branch)

model = Model(inputs=inputs, outputs=[age_output, gender_output])


In [127]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [128]:
# 4. Model Compilation
model.compile(  optimizer='adam', 
                loss={'age': 'mae', 'gender': 'binary_crossentropy'}, 
                metrics={'age': 'mae', 'gender': 'accuracy'},
                loss_weights={'age':1,'gender':99}
            )

In [129]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, dpi=200, rankdir='TB')


In [130]:
# 5. Callbacks
early_stop = EarlyStopping(patience=3, restore_best_weights=True)
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cb = TensorBoard(log_dir=log_dir)



In [133]:
import datetime
from keras.callbacks import TensorBoard

log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_cb = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
# 6. Training
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,
    callbacks=[early_stop, tensorboard_cb]
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs
